In [9]:
import sympy as sp
import numpy as np 
from numpy import radians
from time import perf_counter

In [10]:
num_poles = 2

pole_as = [sp.Symbol(f'a_{i+1}') for i in range(num_poles)]
pole_ls = [sp.Symbol(f'l_{i+1}') for i in range(num_poles)]
pole_ms = [sp.Symbol(f'm_{i+1}') for i in range(num_poles)]
pole_ds = [sp.Symbol(f'd_{i+1}') for i in range(num_poles)]
pole_Js = [sp.Symbol(f'J_{i+1}') for i in range(num_poles)]

m_c = sp.Symbol('m_c')
g = sp.Symbol('g')

In [26]:
t = sp.Symbol("t")
s = sp.Function("s")(t)
d_s = sp.diff(s, t)
dd_s = sp.diff(d_s, t)
thetas = [sp.Function(f"theta_{i+1}")(t) for i in range(num_poles)]
d_thetas = [sp.diff(theta,t) for theta in thetas]
dd_thetas = [sp.diff(d_theta,t) for d_theta in d_thetas]
u = dd_s
torque = sp.Symbol("tau")

In [27]:
pole_pcs = []
for i, (theta, a) in enumerate(zip(thetas, pole_as)):
    prev_1 = 0
    prev_2 = 0
    for prev_l, prev_theta in list(zip(pole_ls, thetas))[:i]:
        prev_1 += -prev_l*sp.sin(prev_theta)
        prev_2 += prev_l*sp.cos(prev_theta)
    pole_pcs.append([s-a*sp.sin(theta)+prev_1, a*sp.cos(theta)+prev_2])

In [28]:
def square_sum(x: tuple):
    result = 0
    for x_i in x:
        result += x_i**2
    return result

T = [1/2*m_c*d_s**2]
for m, pc, J, d_theta in zip(pole_ms, pole_pcs, pole_Js, d_thetas):
    d_pc = [sp.diff(p,t) for p in pc]
    T_p = 1/2*m*square_sum(d_pc) + 1/2*J*d_theta**2
    T.append(T_p)
    print(T_p)
    
V = 0   
for m, pc in zip(pole_ms, pole_pcs  ):
    V += g*m*pc[1]
R = 0
prev_w = 0
for d, d_theta in zip(pole_ds, d_thetas):
    R += 1/2*d*(d_theta-prev_w)**2
    prev_w = d_theta

0.5*J_1*Derivative(theta_1(t), t)**2 + 0.5*m_1*(a_1**2*sin(theta_1(t))**2*Derivative(theta_1(t), t)**2 + (-a_1*cos(theta_1(t))*Derivative(theta_1(t), t) + Derivative(s(t), t))**2)
0.5*J_2*Derivative(theta_2(t), t)**2 + 0.5*m_2*((-a_2*sin(theta_2(t))*Derivative(theta_2(t), t) - l_1*sin(theta_1(t))*Derivative(theta_1(t), t))**2 + (-a_2*cos(theta_2(t))*Derivative(theta_2(t), t) - l_1*cos(theta_1(t))*Derivative(theta_1(t), t) + Derivative(s(t), t))**2)


In [29]:
L = T[0]-V
eqs = []
lh = sp.diff(sp.diff(L, d_s),t) - sp.diff(L, s) + sp.diff(R, d_s)
rh = torque
eqs.append(lh-rh)
for theta, d_theta, T_p in zip(thetas, d_thetas, T[1:]):
    L = T_p-V
    lh = sp.diff(sp.diff(L, d_theta),t) - sp.diff(L, theta) + sp.diff(R, d_theta)
    rh = 0
    eqs.append(lh-rh)

In [30]:
sols = sp.solve(eqs, dd_thetas + [torque])
for key, value in sols.items():
    sols[key] = sp.simplify(value)
    display(sols[key])

m_c*Derivative(s(t), (t, 2))

(a_1*g*m_1*sin(theta_1(t)) + a_1*m_1*cos(theta_1(t))*Derivative(s(t), (t, 2)) - d_1*Derivative(theta_1(t), t) - d_2*Derivative(theta_1(t), t) + d_2*Derivative(theta_2(t), t) + g*l_1*m_2*sin(theta_1(t)))/(J_1 + a_1**2*m_1)

(J_1*a_2*g*m_2*sin(theta_2(t)) + J_1*a_2*l_1*m_2*sin(theta_1(t) - theta_2(t))*Derivative(theta_1(t), t)**2 + J_1*a_2*m_2*cos(theta_2(t))*Derivative(s(t), (t, 2)) + J_1*d_2*Derivative(theta_1(t), t) - J_1*d_2*Derivative(theta_2(t), t) + a_1**2*a_2*g*m_1*m_2*sin(theta_2(t)) + a_1**2*a_2*l_1*m_1*m_2*sin(theta_1(t) - theta_2(t))*Derivative(theta_1(t), t)**2 + a_1**2*a_2*m_1*m_2*cos(theta_2(t))*Derivative(s(t), (t, 2)) + a_1**2*d_2*m_1*Derivative(theta_1(t), t) - a_1**2*d_2*m_1*Derivative(theta_2(t), t) - a_1*a_2*g*l_1*m_1*m_2*sin(theta_1(t))*cos(theta_1(t) - theta_2(t)) + a_1*a_2*l_1*m_1*m_2*sin(theta_1(t) - theta_2(t))*sin(theta_1(t))*Derivative(s(t), (t, 2)) - a_1*a_2*l_1*m_1*m_2*cos(theta_2(t))*Derivative(s(t), (t, 2)) + a_2*d_1*l_1*m_2*cos(theta_1(t) - theta_2(t))*Derivative(theta_1(t), t) + a_2*d_2*l_1*m_2*cos(theta_1(t) - theta_2(t))*Derivative(theta_1(t), t) - a_2*d_2*l_1*m_2*cos(theta_1(t) - theta_2(t))*Derivative(theta_2(t), t) - a_2*g*l_1**2*m_2**2*sin(theta_1(t))*cos(theta_1(t) 

In [31]:
str(sols[dd_thetas[0]])

'(a_1*g*m_1*sin(theta_1(t)) + a_1*m_1*cos(theta_1(t))*Derivative(s(t), (t, 2)) - d_1*Derivative(theta_1(t), t) - d_2*Derivative(theta_1(t), t) + d_2*Derivative(theta_2(t), t) + g*l_1*m_2*sin(theta_1(t)))/(J_1 + a_1**2*m_1)'

In [34]:
sp.sympify(str(sols[dd_thetas[0]]))

(a_1*g*m_1*sin(theta_1(t)) + a_1*m_1*cos(theta_1(t))*Derivative(s(t), (t, 2)) - d_1*Derivative(theta_1(t), t) - d_2*Derivative(theta_1(t), t) + d_2*Derivative(theta_2(t), t) + g*l_1*m_2*sin(theta_1(t)))/(J_1 + a_1**2*m_1)

In [47]:
def sympy2casadi(sympy_expr,sympy_var,casadi_var):
  import casadi
  assert casadi_var.is_vector()
  if casadi_var.shape[1]>1:
    casadi_var = casadi_var.T
  casadi_var = casadi.vertsplit(casadi_var)
  from sympy.utilities.lambdify import lambdify

  mapping = {'ImmutableDenseMatrix': casadi.blockcat,
             'MutableDenseMatrix': casadi.blockcat,
             'Abs':casadi.fabs
            }
  f = lambdify(sympy_var,sympy_expr,modules=[mapping, casadi])
  print(casadi_var)
  return f(*casadi_var)
  

import sympy
x,y = sympy.symbols("x y")
import casadi
X = casadi.SX.sym("x")
Y = casadi.SX.sym("y")

xy = sympy.Matrix([x,y])
e = sympy.Matrix([x*sympy.sqrt(y),sympy.sin(x+y),abs(x-y)])
XY = casadi.vertcat(X,Y)
print(sympy2casadi(e,xy,XY))

[SX(x), SX(y)]
[(x*sqrt(y)), sin((x+y)), fabs((x-y))]


In [33]:
values = np.array([10, 0, radians(0), 0, radians(0), 0, radians(0), 0] + [0])
variables = [s, d_s]
for theta, d_theta in zip(thetas, d_thetas):
    variables.append(theta)
    variables.append(d_theta)
variables.append(u)

start_time = perf_counter()
# substitute the values of the variables into the solutions
sol_sub = sols.copy()
for i in range(100):
    for k, v in sols.items():
        a = v.subs({var: value for var, value in zip(variables, values)})
print(f"Time: {perf_counter()-start_time}")

Time: 0.2281134999357164
